In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Navigate to DDI working directory
%cd drive/MyDrive/DDI
!ls

In [ ]:
# Imports
!pip install pytorch_lightning --quiet
import pytorch_lightning as pl

In [ ]:
from datasets import DDI_DataModule
from models import DDI_DeepDerm

In [ ]:
# RUN EXPERIMENT

# Config parameters
annotation_file = './data/ddi_metadata.csv'
img_dir = './data'
batch_size = 256
num_workers = 4
random_seed = 0
classify_malignant = True
transform = None # TODO: match DDI experiments
skin_tone = None
malignant = None
diseases = None

# PyTorch Lightning
data_module = DDI_DataModule(random_seed,
                              batch_size,
                              num_workers,
                              annotation_file,
                              img_dir,
                              transform,
                              skin_tone,
                              malignant,
                              diseases)
model = DDI_DeepDerm(classify_malignant=classify_malignant)
trainer = pl.Trainer(max_epochs=500) # to match DDI experiments
trainer.fit(model, data_module)